<a href="https://colab.research.google.com/github/AkashW45/DOCPOC/blob/main/DocAI_DOCPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir /content/images

mkdir: cannot create directory ‘/content/images’: File exists


In [3]:
!pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 32.9 MB/s eta 0:00:00


In [4]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.9 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

assert os.environ["GROQ_API_KEY"] is not None
print("✅ Groq API key loaded securely")


✅ Groq API key loaded securely


In [6]:
# ============================================
# Financial Table Image → Clean Markdown Table
# EasyOCR + Groq (Structure Repair)
# ============================================

!pip install easyocr groq

from pathlib import Path
import easyocr
from groq import Groq
from google.colab import userdata
import os

# -----------------------
# CONFIG
# -----------------------
IMAGE_FOLDER = Path("/content/images")
OUTPUT_MD = Path("/content/output.md")

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
client = Groq()

reader = easyocr.Reader(["en"], gpu=True)

md = ["# OCR Output\n"]

# -----------------------
# OCR → raw rows
# -----------------------
def extract_rows(image_path):
    results = reader.readtext(str(image_path))
    rows = {}

    for (bbox, text, conf) in results:
        y = int(sum(p[1] for p in bbox) / 4)
        rows.setdefault(y, []).append(text)

    return [" ".join(rows[y]) for y in sorted(rows)]

# -----------------------
# LLM → table formatter
# -----------------------
def rows_to_markdown(rows_text):
    prompt = f"""
You are a document-structure expert.

Convert the following OCR-extracted financial table rows
into a CLEAN Markdown table with proper | separators.

Rules:
- Detect headers
- Align columns
- Merge wrapped text
- Do NOT invent data
- Output ONLY markdown table

ROWS:
{chr(10).join(rows_text)}
"""

    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()

# -----------------------
# PROCESS IMAGES
# -----------------------
for img in sorted(IMAGE_FOLDER.glob("*")):
    if img.suffix.lower() not in {".jpg", ".png", ".jpeg"}:
        continue

    md.append(f"## {img.name}\n")
    md.append(f"![{img.name}](./images/{img.name})\n")

    rows = extract_rows(img)
    table_md = rows_to_markdown(rows)

    md.append(table_md)
    md.append("\n")

# -----------------------
# SAVE OUTPUT
# -----------------------
OUTPUT_MD.write_text("\n".join(md), encoding="utf-8")
print("✅ Clean Markdown table generated:", OUTPUT_MD)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ Clean Markdown table generated: /content/output.md


In [7]:
from pathlib import Path
from groq import Groq

DOC_PATH = Path("/content/output.md")
document_text = DOC_PATH.read_text(encoding="utf-8")

client = Groq()


In [8]:
SYSTEM_PROMPT = """
You are a Financial Table Reasoning Engine.

The document you receive is a Markdown table extracted from a financial report.

Rules:
- Use ONLY the data in the document.
- Treat each table row as one record.
- Column names are the headers.
- Perform counting, filtering, grouping, and comparison logically.
- Do NOT guess.
- If data is missing, say: "Not found in document."

When a user asks:
- "how many" → COUNT rows
- "list" → RETURN rows
- "details of X" → FILTER rows
- "summarize" → AGGREGATE patterns
"""


In [9]:
def ask_table(question: str) -> str:
    prompt = f"""
DOCUMENT (Markdown table):
{document_text}

QUESTION:
{question}

Answer clearly and precisely.
"""
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1
    )

    return response.choices[0].message.content.strip()


In [10]:
print(ask_table("How many unique MF Holders are there?"))

print(ask_table("List all rows where MF Holder = 'Mr. X'"))

print(ask_table("How many funds have Risk = 'High'?"))

print(ask_table("What is the total Amount (Rs) invested in Income category?"))

print(ask_table("Summarize this portfolio in 5 bullet points"))



APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `openai/gpt-oss-120b` in organization `org_01k0y53bvhfyaa9k8d2wffc02j` service tier `on_demand` on tokens per minute (TPM): Limit 8000, Requested 15054, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
print(ask_table("give me details of 4th row"))